In [ ]:
%pip install -e ../../ag2/

In [1]:
import os

import autogen

config_list = autogen.config_list_from_json(
    "../OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4o"],
    },
)
os.environ["OPENAI_API_KEY"] = config_list[0]["api_key"]

llm_config = {
    "config_list": config_list,
}

/Users/abc/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
from typing import Literal, Optional, Union

from autogen import ConversableAgent
from autogen.agentchat.contrib.rag.parser_utils import docling_parse_docs
from autogen.tools.tool import Tool

parser_tool = Tool(
    name="docling_parse_docs",
    description="Use this tool to parse and understand text.",
    func_or_tool=docling_parse_docs,
)


DEFALT_DOCLING_PARSER_PROMPT = """
You are an expert in parsing and understanding text. You can use this tool to parse various documents and extract information from them.
"""


class ParserAgent(ConversableAgent):
    def __init__(
        self,
        llm_config: Optional[Union[dict, Literal[False]]] = llm_config,
    ):
        super().__init__(
            name="DoclingParserAgent",
            system_message=DEFALT_DOCLING_PARSER_PROMPT,
            human_input_mode="NEVER",
            llm_config=llm_config,
        )

        parser_tool.register_for_llm(self)

In [ ]:
results = docling_parse_docs(
    "/workspaces/ag2/test/agentchat/contrib/graph_rag/Toast_financial_report.pdf", " /workspaces/ag2/output_dir_path"
)

In [ ]:
from autogen import UserProxyAgent

user_agent = UserProxyAgent(
    name="UserAgent",
    human_input_mode="ALWAYS",
)

parser_tool.register_for_execution(user_agent)

parser_agent = ParserAgent()


user_agent.initiate_chat(
    parser_agent,
    message="could you parse /workspaces/ag2/test/agentchat/contrib/graph_rag/Toast_financial_report.pdf and output to /workspaces/ag2/output_dir_path?",
)

In [ ]:
import json
from copy import deepcopy
from enum import Enum
from typing import Any, Dict, List

from pydantic import BaseModel, Field

from autogen import ConversableAgent, SwarmResult, UserProxyAgent, initiate_swarm_chat, register_hand_off
from autogen.agentchat.contrib.rag.docling_doc_ingest_agent import DoclingDocIngestAgent, DoclingMdQueryEngine
from autogen.agentchat.contrib.swarm_agent import AfterWork, AfterWorkOption, OnCondition


class QueryType(Enum):
    RAG_QUERY = "RAG_QUERY"
    COMMON_QUESTION = "COMMON_QUESTION"


class Ingest(BaseModel):
    path_or_url: str = Field(description="The path or URL of the documents to ingest.")


class Query(BaseModel):
    query_type: QueryType = Field(description="The type of query to perform for the Document Agent.")
    query: str = Field(description="The query to perform for the Document Agent.")


class DocumentTask(BaseModel):
    """The structured output format for task decisions."""

    ingestions: list[Ingest] = Field(description="The list of documents to ingest.")
    queries: list[Query] = Field(description="The list of queries to perform.")


class DocumentTriageAgent(ConversableAgent):
    def __init__(
        self,
        llm_config: Dict[str, Any],
        *args,
        **kwargs,
    ):
        # Add the structured message to the LLM configuration
        structured_config_list = deepcopy(llm_config)
        for config in structured_config_list["config_list"]:
            config["response_format"] = DocumentTask

        super().__init__(
            name="DocumentTriageAgent",
            system_message=(
                "You are a document triage agent."
                "You are responsible for deciding what type of task to perform from user requests."
                "When user uploads new documents or provide links of documents, you should add Ingest task to DocumentTask."
                "When user asks common questions, you should add 'COMMON_QUESTION' Query task to DocumentTask."
                "When user asks questions about information from existing documents, you add 'RAG_QUERY' Query task to DocumentTask."
            ),
            human_input_mode="NEVER",
            llm_config=structured_config_list,
        )


triage_agent = DocumentTriageAgent(llm_config=llm_config)


def initiate_tasks(ingestions: list[Ingest], queries: list[Query], context_variables: dict) -> SwarmResult:
    print("initiate_tasks context_variables", context_variables)
    if "TaskInitiated" in context_variables:
        return SwarmResult(values="Task already initiated", context_variables=context_variables)
    context_variables["DocumentsToIngest"] = ingestions
    context_variables["QueriesToRun"] = [query for query in queries]
    context_variables["TaskInitiated"] = True
    return SwarmResult(
        values="Updated context variables with task decisions",
        context_variables=context_variables,
        agent="TaskManagerAgent",
    )


task_manager_agent = ConversableAgent(
    name="TaskManagerAgent",
    system_message="""
    You are a task manager agent. You would do the following:
    1. You update the context variables based on the task decisions (DocumentTask) from the DocumentTriageAgent.
    i.e. output
    {
        "ingestions": [
            {
                "path_or_url": "path_or_url"
            }
        ],
        "queries": [
            {
                "query_type": "RAG_QUERY",
                "query": "query"
            }
        ],
        "query_results": [
            {
                "query": "query",
                "result": "result"
            }
        ]
    }
    2. You would hand off control to the appropriate agent based on the context variables.
    """,
    llm_config=llm_config,
    functions=[initiate_tasks],
)

register_hand_off(
    agent=triage_agent,
    hand_to=[
        # ON_CONDITION(task_manager_agent, "After output task desicisions, transfer to task manager agent"),
        AfterWork(task_manager_agent),
    ],
)

query_engine = DoclingMdQueryEngine()
data_ingestion_agent = DoclingDocIngestAgent(llm_config=llm_config, query_engine=query_engine)


def execute_rag_query(context_variables: dict) -> SwarmResult:
    query = context_variables["QueriesToRun"][0]["query"]
    answer = query_engine.query(query)
    context_variables["QueriesToRun"].pop(0)
    context_variables["CompletedTaskCount"] += 1
    context_variables["QueryResults"].append({"query": query, "result": answer})
    return SwarmResult(values=answer, context_variables=context_variables)


query_agent = ConversableAgent(
    name="QueryAgent",
    system_message="You are a query agent. You answer the user's questions only using the query function provided to you.",
    llm_config=llm_config,
    functions=[execute_rag_query],
)

summary_agent = ConversableAgent(
    name="SummaryAgent",
    system_message="You are a summary agent. You would generate a summary of all completed tasks and  answer the user's questions.",
    llm_config=llm_config,
)


def has_ingest_tasks(agent: ConversableAgent, messages: List[Dict[str, Any]]) -> bool:
    print("context_variables", agent._context_variables)
    return len(agent.get_context("DocumentsToIngest")) > 0


def has_query_tasks(agent: ConversableAgent, messages: List[Dict[str, Any]]) -> bool:
    print("context_variables", agent._context_variables)
    if len(agent.get_context("DocumentsToIngest")) > 0:
        return False
    return len(agent.get_context("QueriesToRun")) > 0


def summary_task(agent: ConversableAgent, messages: List[Dict[str, Any]]) -> bool:
    return (
        len(agent.get_context("DocumentsToIngest")) == 0
        and len(agent.get_context("QueriesToRun")) == 0
        and agent.get_context("CompletedTaskCount")
    )


register_hand_off(
    agent=task_manager_agent,
    hand_to=[
        OnCondition(
            data_ingestion_agent,
            "If there are any DocumentsToIngest in context variables, transfer to data ingestion agent",
            available=has_ingest_tasks,
        ),
        OnCondition(
            query_agent,
            "If there are any QueriesToRun in context variables and no DocumentsToIngest, transfer to query_agent",
            available=has_query_tasks,
        ),
        OnCondition(
            summary_agent,
            "If there are no DocumentsToIngest or QueriesToRun in context variables, transfer to summary_agent",
            available=summary_task,
        ),
        # AfterWork(AfterWorkOption.TERMINATE),
    ],
)

register_hand_off(
    agent=data_ingestion_agent,
    hand_to=[
        AfterWork(task_manager_agent),
    ],
)
register_hand_off(
    agent=query_agent,
    hand_to=[
        AfterWork(task_manager_agent),
    ],
)

user_agent = UserProxyAgent(
    name="UserAgent",
    system_message="A human admin.",
    human_input_mode="ALWAYS",
)

register_hand_off(
    agent=summary_agent,
    hand_to=[
        AfterWork(AfterWorkOption.TERMINATE),
    ],
)
# chat_result = user_agent.initiate_chat(
#     triage_agent,
#     message="could you ingest /workspaces/ag2/test/agentchat/contrib/graph_rag/Toast_financial_report.pdf and summarise the FY2024 financials?",
# )

# print(chat_result)

In [ ]:
context_variables = {
    "CompletedTaskCount": 0,
    "DocumentsToIngest": [],
    "QueriesToRun": [],
    "QueryResults": [],
}

# for agent in [triage_agent, task_manager_agent, data_ingestion_agent, query_agent, summary_agent]:
#     agent.reset()
#     agent._context_variables = None
#     print(agent.name, agent._context_variables)

In [ ]:
chat_result, context_variables, last_speaker = initiate_swarm_chat(
    initial_agent=triage_agent,  # Starting agent
    agents=[triage_agent, task_manager_agent, data_ingestion_agent, query_agent, summary_agent],
    messages="could you ingest /workspaces/ag2/test/agentchat/contrib/graph_rag/Toast_financial_report.pdf and summarize the FY2024 financials?",
    context_variables=context_variables,  # Context
    after_work=AfterWork(AfterWorkOption.TERMINATE),  # Swarm-level after work hand off
)

print(f"Context Variables:\n{json.dumps(context_variables, indent=2)}")

In [5]:
import os

print(os.getcwd())

/Users/abc/workspace/ag2ai/ag2/notebook


In [3]:
from autogen.agentchat.contrib.rag.document_agent import DocumentAgent

document_agent = DocumentAgent(llm_config=llm_config)
document_agent.run(
    "could you ingest ../test/agentchat/contrib/graph_rag/Toast_financial_report.pdf and summarize the FY2024 financials?"
)

user (to Document_Agent):

could you ingest ../test/agentchat/contrib/graph_rag/Toast_financial_report.pdf and summarize the FY2024 financials?

--------------------------------------------------------------------------------
_User (to chat_manager):

could you ingest ../test/agentchat/contrib/graph_rag/Toast_financial_report.pdf and summarize the FY2024 financials?

--------------------------------------------------------------------------------

Next speaker: DocumentTriageAgent

DocumentTriageAgent (to chat_manager):

{"ingestions":[{"path_or_url":"../test/agentchat/contrib/graph_rag/Toast_financial_report.pdf"}],"queries":[{"query_type":"RAG_QUERY","query":"Summarize the FY2024 financials from the Toast financial report."}]}

--------------------------------------------------------------------------------

Next speaker: TaskManagerAgent

context_variables {'CompletedTaskCount': 0, 'DocumentsToIngest': [], 'QueriesToRun': [], 'QueryResults': []}
context_variables {'CompletedTaskCoun

INFO:autogen.agentchat.contrib.rag.document_utils:Error when checking if ../test/agentchat/contrib/graph_rag/Toast_financial_report.pdf is a valid URL: Invalid URL.
INFO:autogen.agentchat.contrib.rag.document_utils:Detected file. Returning file path...
INFO:docling.document_converter:Going to convert document batch...
/Users/abc/anaconda3/lib/python3.11/site-packages/docling/models/easyocr_model.py:58: UserWarning: Deprecated field. Better to set the `accelerator_options.device` in `pipeline_options`. When `use_gpu and accelerator_options.device == AcceleratorDevice.CUDA` the GPU is used to run EasyOCR. Otherwise, EasyOCR runs in CPU.
  warnings.warn(
INFO:docling.utils.accelerator_utils:Accelerator device: 'mps'
INFO:docling.utils.accelerator_utils:Accelerator device: 'mps'
INFO:docling.pipeline.base_pipeline:Processing document Toast_financial_report.pdf
INFO:docling.document_converter:Finished converting document Toast_financial_report.pdf in 15.39 sec.
INFO:autogen.agentchat.contri

docling ingest: {'CompletedTaskCount': 1, 'DocumentsToIngest': [], 'QueriesToRun': [{'query_type': 'RAG_QUERY', 'query': 'Summarize the FY2024 financials from the Toast financial report.'}], 'QueryResults': [], 'TaskInitiated': True} 
 {'CompletedTaskCount': 1, 'DocumentsToIngest': [], 'QueriesToRun': [{'query_type': 'RAG_QUERY', 'query': 'Summarize the FY2024 financials from the Toast financial report.'}], 'QueryResults': [], 'TaskInitiated': True}
_Swarm_Tool_Executor (to chat_manager):

***** Response from calling tool (call_xUMm6Cv9f0Lg5F7jJ2cshGMK) *****
Data Ingestion Task Completed for ../test/agentchat/contrib/graph_rag/Toast_financial_report.pdf
**********************************************************************

--------------------------------------------------------------------------------

Next speaker: TaskManagerAgent

context_variables {'CompletedTaskCount': 1, 'DocumentsToIngest': [], 'QueriesToRun': [{'query_type': 'RAG_QUERY', 'query': 'Summarize the FY2024 financ

ChatResult(chat_id=None, chat_history=[{'content': 'could you ingest ../test/agentchat/contrib/graph_rag/Toast_financial_report.pdf and summarize the FY2024 financials?', 'role': 'assistant', 'name': 'user'}, {'content': 'Here is the summary of the completed tasks:\n\n1. The document `../test/agentchat/contrib/graph_rag/Toast_financial_report.pdf` was successfully ingested.\n2. A query was executed to summarize the FY2024 financials of Toast, Inc.\n\n   - For the fiscal year 2024, Toast, Inc. reported total revenue of $3,622 million.\n   - The major contributions to revenue were from financial technology solutions ($2,963 million) and subscription services ($506 million).\n   - The cost of revenue was $2,765 million, resulting in a gross profit of $857 million.\n   - Operating expenses amounted to $873 million, which included sales and marketing ($340 million), research and development ($258 million), and general and administrative costs ($229 million), along with $46 million in restru

In [4]:
from autogen.agentchat.contrib.rag.document_agent import DocumentAgent

document_agent = DocumentAgent(llm_config=llm_config)
document_agent.run(
    "could you read 'https://www.independent.co.uk/space/earth-core-inner-shape-change-b2695585.html' and summarize the article?"
)

user (to Document_Agent):

could you read 'https://www.independent.co.uk/space/earth-core-inner-shape-change-b2695585.html' and summarize the article?

--------------------------------------------------------------------------------
_User (to chat_manager):

could you read 'https://www.independent.co.uk/space/earth-core-inner-shape-change-b2695585.html' and summarize the article?

--------------------------------------------------------------------------------

Next speaker: DocumentTriageAgent

DocumentTriageAgent (to chat_manager):

{"ingestions":[{"path_or_url":"https://www.independent.co.uk/space/earth-core-inner-shape-change-b2695585.html"}],"queries":[]}

--------------------------------------------------------------------------------

Next speaker: TaskManagerAgent

context_variables {'CompletedTaskCount': 0, 'DocumentsToIngest': [], 'QueriesToRun': [], 'QueryResults': []}
context_variables {'CompletedTaskCount': 0, 'DocumentsToIngest': [], 'QueriesToRun': [], 'QueryResults': []

INFO:autogen.agentchat.contrib.rag.document_utils:Detected URL. Downloading content...
INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/abc/.wdm/drivers/chromedriver/mac64/133.0.6943.98/chromedriver-mac-arm64/chromedriver] found in cache
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.pipeline.base_pipeline:Processing document earth-core-inner-shape-change-b2695585.html
INFO:docling.document_converter:Finished converting document earth-core-inner-shape-change-b2695585.html in 0.02 sec.
INFO:autogen.agentchat.contrib.rag.parser_utils:Document converted in 0.02 seconds.
INFO:autogen.agentchat.contrib.rag.docling_query_engine:Collection docling-parsed-docs was created in the database.
INFO:autogen.agentchat.contrib.rag.docling_query_engine:Loading input doc: /Users/abc/workspace/ag2ai/ag2/notebook/parsed_docs/earth-core-inner

docling ingest: {'CompletedTaskCount': 1, 'DocumentsToIngest': [], 'QueriesToRun': [], 'QueryResults': [], 'TaskInitiated': True} 
 {'CompletedTaskCount': 1, 'DocumentsToIngest': [], 'QueriesToRun': [], 'QueryResults': [], 'TaskInitiated': True}
_Swarm_Tool_Executor (to chat_manager):

***** Response from calling tool (call_enIa1Wm4uXrdk4KDiFyavRmq) *****

**********************************************************************

--------------------------------------------------------------------------------

Next speaker: TaskManagerAgent

context_variables {'CompletedTaskCount': 1, 'DocumentsToIngest': [], 'QueriesToRun': [], 'QueryResults': [], 'TaskInitiated': True}
context_variables {'CompletedTaskCount': 1, 'DocumentsToIngest': [], 'QueriesToRun': [], 'QueryResults': [], 'TaskInitiated': True}

>>>>>>>> USING AUTO REPLY...
TaskManagerAgent (to chat_manager):

I have successfully ingested the document from the URL provided. Now I will proceed to summarize the document.
***** Suggest

ChatResult(chat_id=None, chat_history=[{'content': "could you read 'https://www.independent.co.uk/space/earth-core-inner-shape-change-b2695585.html' and summarize the article?", 'role': 'assistant', 'name': 'user'}, {'content': 'I have read the article from The Independent titled "Earth’s core has changed its shape, say scientists" and here is the summary:\n\nRecent studies have revealed that the Earth\'s inner core is not as static as previously thought. Scientists from China have observed that the solid iron core, which is about the size of Pluto, has undergone changes both in its shape and rotation. These changes are believed to influence Earth’s magnetic field and are connected to phenomena such as the variations in the planet\'s day length and magnetic field anomalies. The research suggests that the core\'s rotation is slowing down relative to the Earth\'s surface, and it could even oscillate, going slightly faster or slower over a decade. The precise reasons for these changes and